## 前回のコードのまとめ
本日の演習では、こちらの学習コードを使用します。

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ArtIC-TITECH/b3-proj-2023/blob/feature/class_02/docs/class_02.ipynb)

In [ ]:
## Pytorch関連ライブラリ
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
## Please update the path to your own directory.
# path=/path/to/your_own  # Uncomment this line
path = '../../work/data/cifar10'

## Define Augmentation
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
     ])

batch_size = 128

trainset = torchvision.datasets.CIFAR10(root=path, train=True,
                                        download=True, transform=transform)

n_samples = len(trainset)
trainsize = int(n_samples * 0.8)

trainsubset, validsubset = torch.utils.data.random_split(trainset, [trainsize, n_samples-trainsize])

trainloader = torch.utils.data.DataLoader(trainsubset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

valloader = torch.utils.data.DataLoader(validsubset, batch_size=batch_size, 
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root=path, train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.bn1 = nn.BatchNorm2d(6)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.bn2 = nn.BatchNorm2d(16)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net().to('cuda:0')

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-5)

In [ ]:
def accuracy(loader, model):
    total = 0
    correct = 0
    net.eval()
    with torch.no_grad():
        for data in loader:
            images, labels = data[0].to('cuda:0'), data[1].to('cuda:0')
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    net.train()
    return 100.0 * correct / total

def accuracy_batch(outputs, labels):
    total = 0
    correct = 0
    total = labels.size(0)
    _, predicted = torch.max(outputs.data, 1)
    correct = (predicted == labels).sum().item()
    return 100 * correct / total

In [ ]:
net.train()
for epoch in range(50):
    running_loss = 0.
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data[0].to('cuda:0'), data[1].to('cuda:0')

        # 1. forward
        outputs = net(inputs)
        # 2. compute loss
        loss = criterion(outputs, labels)
        
        # 3. reset parameter gradient
        optimizer.zero_grad()

        # 4. backward
        loss.backward()

        # 5. update parameters
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 100 == 99:
            ## Calculate validation accuracy
            val_acc = accuracy(valloader, net)

            ## Calculate batch accuracy
            batch_acc = accuracy_batch(outputs=outputs, labels=labels)

            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}, validation accuracy: {val_acc}, batch accuracy: {batch_acc}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
accuracy(testloader, net)

In [ ]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

## 前回の課題

`Net`クラスを修正して以下で定義されるモデルを実装してください。\
Conv2dは全て`padding=0`、`stride=1`で、MaxPool2dは`stride=2`で実装してください。\
それぞれの層のカーネルサイズは、テーブルの`Kernel Shape`を元に決定してください。\
それぞれの層のチャネル数は、入力チャネル、出力チャネルを参考に実装してください。\

```
===================================================================================================================
Layer (type:depth-idx)                   Input Shape               Output Shape              Kernel Shape
===================================================================================================================
Net                                      [4, 3, 32, 32]            [4, 10]                   --
├─Conv2d: 1-1                            [4, 3, 32, 32]            [4, 6, 28, 28]            [5, 5]
│    └─weight                                                                                [3, 6, 5, 5]
│    └─bias                                                                                  [6]
|-ReLU                                   [4, 6, 28, 28]            [4, 6, 28, 28]
├─MaxPool2d: 1-2                         [4, 6, 28, 28]            [4, 6, 14, 14]            2
├─Conv2d: 1-3                            [4, 6, 14, 14]            [4, 16, 10, 10]           [5, 5]
│    └─weight                                                                                [6, 16, 5, 5]
│    └─bias                                                                                  [16]
|-ReLU                                   [4, 16, 10, 10]           [4, 16, 10, 10]
├─MaxPool2d: 1-4                         [4, 16, 10, 10]           [4, 16, 5, 5]             2
├─Linear: 1-5                            [4, 400]                  [4, 120]                  --
│    └─weight                                                                                [400, 120]
│    └─bias                                                                                  [120]
|-ReLU                                   [4, 120]                  [4, 120]
├─Linear: 1-6                            [4, 120]                  [4, 84]                   --
│    └─weight                                                                                [120, 84]
│    └─bias                                                                                  [84]
|-ReLU                                   [4, 84]                   [4, 84]
├─Linear: 1-7                            [4, 84]                   [4, 10]                   --
│    └─weight                                                                                [84, 10]
│    └─bias                                                                                  [10]
===================================================================================================================
```

### 結果の確認

はじめに自身で定義したモデルが一致しているか確認したいと思います。\
まずは、Netを定義しているセルに移動して先週自分で実装したモデルと入れ替えてください。

今回は出力が一致しているか確認するための検証データを用意しているのでそちらと結果が一致するか確認してみましょう。

In [ ]:
## 検証用のデータのダウンロード
!curl -L -o model_out.pt https://github.com/ArtIC-TITECH/b3-proj-2023/raw/main/resources/model_out.pt

下のセルを実行して何もエラーが発生しなければモデルが正しく実装できていることになります。\
結果が一致しない場合はモデルが正しく定義できているかもう一度確認してみましょう。\
(乱数のシードを一致させることで入出力の再現をしているため、モデルを定義する際の順序が異なると結果が一致しない可能性があります。層を実行する順番と定義する順番が一致するように注意してください。)

In [ ]:
torch.manual_seed(100)

input = torch.rand((1, 3, 32, 32))
net = Net()
output = net(input)

valid = torch.load('./model_out.pt')

assert torch.isclose(valid, output).all(), "Validation is failed"

### 解答

本日の演習ではこちらのモデルを使用するため、結果が一致しない場合は`Net`の定義をこちらのモデルに書き換えてください。

```python
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
```

## Tensorboardを利用したモデルの可視化

本日は`Tensorboard`というライブラリを使用した学習経過の視覚化を行います。\
学習過程を観察することでモデルの精度を上げるためのヒントを得られることがあります。

まず、`Tensorboard`をGoogle Colabo上で利用可能にするには下記のコマンドを実行する必要があります。
pytorchのライブラリをimportするセルの上に新しいセルを作成して、下記のコマンドをコピーした後に実行してください。

### writerの定義

tensorboardのSummaryWriterというクラスを用いてwriterを初期化します。
下のセルにおいて、`path`を自身のGoogle Driveのパスに変更してください。

```python
from torch.utils.tensorboard import SummaryWriter
import os

## Put your google drive path
path = ''
run_name = 'runs/cifar10_experiments1'
run_dir = os.path.join(path, run_name)

writer = SummaryWriter(run_dir)
```

別の実験を行う場合は`run_name`を`runs/cifar10_experimentsN`等に変更するようにしてください。
実験を行いやすいようwriterを定義するセルの場所は適宜変更してください。

In [ ]:
## Define writer
from torch.utils.tensorboard import SummaryWriter
import os

## Put your google drive path
path = ''
run_name = 'runs/cifar10_experiments1'
run_dir = os.path.join(path, run_name)

writer = SummaryWriter(run_dir)

### Tensorboardを使ったモデル構造の出力

`writer.add_xxx`という関数を呼び出すことでwriterにデータを記録することができます。\
(xxxは記録したいデータによって変わります。)

まず初めに、モデル構造をtensorboardに記録してみましょう。
次のセルを実行してみてください。

`add_graph`という関数はモデル構造をtensorboardに記録するための関数で、第一引数がモデル、第二引数がモデルへの入力となります。\
前回も説明した通りCIFAR-10は$3\times 32\times 32$のテンソルなのでそれと同じ形状の乱数をここでは入力しています。\
(もちろんデータローダーから読み出した画像データを直接入力に使用しても問題ありません。)

In [ ]:
writer.add_graph(net, torch.randn(1, 3, 32, 32))

上のセルを実行したら次のセルを実行してtensorboardを起動してください。

In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./drive/My\ Drive/Colab\ Notebooks/b3_proj_2023/runs